In [82]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option("display.width", 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [83]:
# 데이터셋 로드
df = pd.read_csv("data/movies_metadata_with_weather.csv", index_col=0)
df.head()

/var/folders/t7/qdmvn4v5227c20lm4jd3098m0000gn/T/ipykernel_45783/221975465.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/movies_metadata_with_weather.csv", index_col=0)


,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,combined,weather
adult,,,,,,,,,,,,,,,,,,,,,,,,,
False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"Toy Story [{'id': 16, 'name': 'Animation'}, {'...",Clouds
False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"Jumanji [{'id': 12, 'name': 'Adventure'}, {'id...",Rain
False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"Grumpier Old Men [{'id': 10749, 'name': 'Roman...",Thunderstorm
False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"Waiting to Exhale [{'id': 35, 'name': 'Comedy'...",Thunderstorm
False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"Father of the Bride Part II [{'id': 35, 'name'...",Rain


In [84]:
null_counts = df.isnull().sum()

# 결과 출력
print("각 속성별 결측치 개수:")
print(null_counts)

각 속성별 결측치 개수:
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
combined                     0
weather                      0
dtype: int64


In [85]:
df["weather"].value_counts()

weather
Thunderstorm    24550
Rain             7285
Snow             6318
Clouds           4153
Clear            1400
Fog               836
Mist              493
Drizzle           431
Name: count, dtype: int64

In [86]:
df = df.dropna(subset=['title', 'overview'])

In [87]:
null_counts = df.isnull().sum()

# 결과 출력
print("각 속성별 결측치 개수:")
print(null_counts)

각 속성별 결측치 개수:
belongs_to_collection    40075
budget                       0
genres                       0
homepage                 36745
id                           0
imdb_id                     15
original_language           10
original_title               0
overview                     0
popularity                   0
poster_path                343
production_companies         0
production_countries         0
release_date                71
revenue                      0
runtime                      0
spoken_languages             0
status                      65
tagline                  24102
title                        0
video                        0
vote_average                 0
vote_count                   0
combined                     0
weather                      0
dtype: int64


In [88]:
df["weather"].value_counts()

weather
Thunderstorm    23941
Rain             7138
Snow             6211
Clouds           4086
Clear            1389
Fog               829
Mist              489
Drizzle           423
Name: count, dtype: int64

In [123]:
# 날씨 정보 가져오기 (OpenWeatherMap)
def get_weather(city_name):
    api_key = " "  # OpenWeatherMap API 키를 추가

    lat_url = "http://api.openweathermap.org/geo/1.0/direct?"
    lat_response = requests.get(lat_url, params={"q": city_name, "appid": api_key})
    lat_data = lat_response.json()

    if lat_data:
        latitude = lat_data[0]["lat"]
        longitude = lat_data[0]["lon"]

        base_url = "https://api.openweathermap.org/data/2.5/weather?"
        complete_url = base_url + f"lat={latitude}&lon={longitude}&appid={api_key}&units=metric"

        response = requests.get(complete_url)
        weather_data = response.json()

        if "weather" in weather_data:
            main_info = weather_data["weather"][0]["main"]
            description = weather_data["weather"][0]["description"]
            temperature = weather_data["main"]["temp"]
            humidity = weather_data["main"]["humidity"]
            wind_speed = weather_data["wind"]["speed"]

            print(f"##### {city}의 날씨 정보 #####")
            print(f"날씨: {main_info}")
            print(f"하늘: {description}")
            print(f"기온: {temperature}°C")
            print(f"습도: {humidity}%")
            print(f"바람: {wind_speed}m/s")
            print(f"###########################")
            return main_info

        else:
            print("날씨 정보를 찾을 수 없습니다.")
               
    else:
        print("도시를 찾을 수 없습니다.")
    

city = input("도시를 입력해주세요.: ")
city_weather = get_weather(city)

#print(city, city_weather)

##### 서울의 날씨 정보 #####
날씨: Clear
하늘: clear sky
기온: 22.97°C
습도: 38%
바람: 3.6m/s
###########################


In [124]:
# 위에서 입력한 도시의 날씨 기반 영화 추천
def get_movies_by_weather(dataframe, weather_col, tf_idf_col, weather_variable, num_of_recommend):

    # 입력된 도시의 날씨 기반 영화 필터링
    filtered_movies = dataframe[(dataframe[weather_col] == weather_variable)]

    # 추천할 영화 수
    num_movies_to_recommend = num_of_recommend

    # 필터링된 영화 수 확인
    num_movies_available = len(filtered_movies)

    # TF-IDF 벡터화
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(filtered_movies[tf_idf_col])

    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # 랜덤 인덱스 생성
    random_indices = np.random.choice(num_movies_available, size=num_movies_to_recommend, replace=False)

    # 유사도에 따라 영화 선택
    movie_indices = cosine_sim[0].argsort()

    # 임의의 인덱스 사용하여 영화 선택
    top_movies_indices = movie_indices[random_indices][::-1]

    # 최상위 영화 선택
    top_movies = filtered_movies.iloc[top_movies_indices]

    # 평점에 따라 영화 정렬
    top_movies = top_movies.sort_values("vote_average", ascending=False).reset_index(drop=True) #drop=True

    # 추천 영화 출력
    return print(f"오늘 {city} 날씨는 {city_weather} 입니다. 날씨와 어울리는 영화를 추천해드립니다.:" "\n",
                 top_movies[
                     ['title', 'release_date', 'genres', 'overview', 'vote_average', "weather"]])


get_movies_by_weather(df, "weather", "overview", city_weather, 20)

오늘 서울 날씨는 Clear 입니다. 날씨와 어울리는 영화를 추천해드립니다.:
                                                 title release_date                                             genres                                           overview  vote_average weather
0                                  Miss You Can Do It   2013-06-24  [{'id': 99, 'name': 'Documentary'}, {'id': 107...  Miss You Can Do It chronicles Abbey Curran, Mi...           8.5   Clear
1                               The Enemies of Reason   2007-01-01                [{'id': 99, 'name': 'Documentary'}]  Documentary written and presented by scientist...           7.5   Clear
2                    Monster High Great Scarrier Reef   2016-02-12  [{'id': 10751, 'name': 'Family'}, {'id': 16, '...  Join the Monster High ghouls on an all new aqu...           7.2   Clear
3                                  The Windmill Movie   2008-10-05                [{'id': 99, 'name': 'Documentary'}]  Filmmaker Richard P. Rogers tried for twenty y...           7.0   Clear
